In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np
!pip install folium
import folium 

In [2]:
#hiden cell
CLIENT_ID = 'DPBYY4JUY3DU20ALPSUV4ONY2K1GOJJKJ1NIHBB32XEMOVYY' # your Foursquare ID
CLIENT_SECRET = '1MV443TYEP4HUO0WDUW5NQ5W10L2Y4G05NWG11WIR3NUGC5B' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 40
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DPBYY4JUY3DU20ALPSUV4ONY2K1GOJJKJ1NIHBB32XEMOVYY
CLIENT_SECRET:1MV443TYEP4HUO0WDUW5NQ5W10L2Y4G05NWG11WIR3NUGC5B


In [3]:
import requests

request_parameters = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "v": '20180605',
    "section": "Library",
    "categoryId":"4bf58dd8d48988d12f941735",
    "near": "yokohama",
    "radius": 5000,
    "limit": 1000}

data = requests.get("https://api.foursquare.com/v2/venues/explore", params=request_parameters)

In [4]:
d = data.json()["response"]
d.keys()

dict_keys(['geocode', 'headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'query', 'totalResults', 'suggestedBounds', 'groups'])

In [5]:
d["headerLocationGranularity"], d["headerLocation"], d["headerFullLocation"]

('city', 'Yokohama', 'Yokohama')

In [6]:
d["suggestedBounds"], d["totalResults"]

({'ne': {'lat': 35.485704465587176, 'lng': 139.66877467365984},
  'sw': {'lat': 35.420427029295155, 'lng': 139.59300943455438}},
 29)

In [7]:
d["geocode"]

{'what': '',
 'where': 'yokohama',
 'center': {'lat': 35.44778, 'lng': 139.6425},
 'displayString': 'Yokohama, Kanagawa, Japan',
 'cc': 'JP',
 'geometry': {'bounds': {'ne': {'lat': 35.67284095920208,
    'lng': 139.74687200002816},
   'sw': {'lat': 35.12857911264956, 'lng': 138.916107002802}}},
 'slug': 'yokohama-shi',
 'longId': '72057594039776290'}

In [8]:
d["groups"][0].keys()

dict_keys(['type', 'name', 'items'])

In [9]:
d["groups"][0]["type"], d["groups"][0]["name"]

('Recommended Places', 'recommended')

In [10]:
items = d["groups"][0]["items"]
print("number of items: %i" % len(items))
items[0]

number of items: 10


{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4b5a8fbff964a52068cb28e3',
  'name': '横浜市中央図書館',
  'location': {'address': '西区老松町1',
   'lat': 35.447413,
   'lng': 139.626411,
   'labeledLatLngs': [{'label': 'display',
     'lat': 35.447413,
     'lng': 139.626411}],
   'postalCode': '220-0032',
   'cc': 'JP',
   'city': '横浜市',
   'state': '神奈川県',
   'country': '日本',
   'formattedAddress': ['西区老松町1', '横浜市, 神奈川県', '220-0032', '日本']},
  'categories': [{'id': '4bf58dd8d48988d12f941735',
    'name': 'Library',
    'pluralName': 'Libraries',
    'shortName': 'Library',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/library_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-0-4b5a8fbff964a52068cb28e3-0'}

In [11]:
df_raw = []
for item in items:
    venue = item["venue"]
    categories, uid, name, location = venue["categories"], venue["id"], venue["name"], venue["location"]
    print(location)
    assert len(categories) == 1
    shortname = categories[0]["shortName"]
    address =  ''
    if hasattr(location, 'address'):
      address = location['address']
    if not "postalCode" in location:
        continue
    postalcode = location["postalCode"]
    lat = location["lat"]
    lng = location["lng"]
    datarow = (uid, name, shortname, address, postalcode, lat, lng)
    df_raw.append(datarow)
df = pd.DataFrame(df_raw, columns=["uid", "name", "shortname", "address", "postalcode", "lat", "lng"])
print("found %i Library" % len(df))
df.head(30)

{'address': '西区老松町1', 'lat': 35.447413, 'lng': 139.626411, 'labeledLatLngs': [{'label': 'display', 'lat': 35.447413, 'lng': 139.626411}], 'postalCode': '220-0032', 'cc': 'JP', 'city': '横浜市', 'state': '神奈川県', 'country': '日本', 'formattedAddress': ['西区老松町1', '横浜市, 神奈川県', '220-0032', '日本']}
{'address': '西区紅葉ケ丘9-2', 'crossStreet': '紅葉坂', 'lat': 35.452192197584424, 'lng': 139.62621994265197, 'labeledLatLngs': [{'label': 'display', 'lat': 35.452192197584424, 'lng': 139.62621994265197}], 'postalCode': '220-8585', 'cc': 'JP', 'city': '横浜市', 'state': '神奈川県', 'country': '日本', 'formattedAddress': ['西区紅葉ケ丘9-2 (紅葉坂)', '横浜市, 神奈川県', '220-8585', '日本']}
{'address': '西区みなとみらい3-4-1', 'crossStreet': '横浜美術館 3F', 'lat': 35.45661675921584, 'lng': 139.63105648756027, 'labeledLatLngs': [{'label': 'display', 'lat': 35.45661675921584, 'lng': 139.63105648756027}], 'postalCode': '220-0012', 'cc': 'JP', 'city': '横浜市', 'state': '神奈川県', 'country': '日本', 'formattedAddress': ['西区みなとみらい3-4-1 (横浜美術館 3F)', '横浜市, 神奈川県', '22

,uid,name,shortname,address,postalcode,lat,lng
0,4b5a8fbff964a52068cb28e3,横浜市中央図書館,Library,,220-0032,35.447413,139.626411
1,4b5ba63af964a520980d29e3,神奈川県立図書館,Library,,220-8585,35.452192,139.626220
2,4b67aff1f964a520d45a2be3,Art Information Center (美術情報センター),Library,,220-0012,35.456617,139.631056
3,553629a0498eaf2e168ba33b,City of Yokohama Municipal Archives Reference ...,Library,,220-0032,35.447413,139.626411
4,57cfbe49cd102d0ceb99971c,希望-M・M (けいゆう病院 患者図書室),Library,,220-0012,35.459370,139.633350
5,4b6d007cf964a52070622ce3,横浜市中図書館,Library,,231-0821,35.423394,139.665331
6,4b63b1c2f964a520908c2ae3,横浜市立 神奈川図書館,Library,,221-0063,35.482737,139.635229
7,4b6bff66f964a520ea1e2ce3,保土ケ谷図書館,Library,,240-0006,35.457126,139.596591
8,4b797255f964a520e6f92ee3,横浜市南図書館,Library,,232-0067,35.423897,139.596453
9,4b071c79f964a52076f722e3,Yokohama International School,High School,,231-0862,35.439012,139.653553


In [12]:
yokohama_center = d["geocode"]["center"]
yokohama_center

{'lat': 35.44778, 'lng': 139.6425}

In [13]:
from folium import plugins


map_yokohama= folium.Map(location=[35.44778,  139.6425], zoom_start=14)

def add_markers(df):
    for (j, row) in df.iterrows():
        label = folium.Popup(row["name"], parse_html=True)
        folium.CircleMarker(
            [row["lat"], row["lng"]],
            radius=2,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_yokohama)

add_markers(df)
hm_data = df[["lat", "lng"]].to_numpy().tolist()
map_yokohama.add_child(plugins.HeatMap(hm_data))

map_yokohama